In [1]:
%matplotlib widget
from bmcs_shell.api import WBCell5Param, WBTessellation5PV2, WBNumTessellation, WBNumTessellationInvest, WBTessellationBase, WBNumTessellationBase, WBCell5ParamV2, WBTessellation4P, WBCell4Param
import numpy as np
import matplotlib.pyplot as plt

## Generate, extrude and subtract the geometry using gmsh

In [2]:
wbt = WBTessellation4P(a = 50, b = 50, c=50, gamma=np.pi/2-1e-3, wireframe_width=1, n_phi_plus=2, n_x_plus=2)
# wbt = WBTessellation4P(a = 50, b = 50, c=50, gamma=np.pi/2-1e-3, wireframe_width=1, n_phi_plus=3, n_x_plus=2)
wbt.interact()

C:\Users\hspartali\Anaconda3\envs\bmcs_env\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


In [3]:
I_Fi = wbt.I_Fi
# I_Fi

In [4]:
X_Ia = np.round(wbt.X_Ia, 0)
# X_Ia

### Basic mesh generation:

In [36]:
# See https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorials/python/t11.py
# dimTag means tuple of (dimention, tag). Where tag is like an ID

import gmsh
import sys
import numpy as np
import bmcs_utils.api as bu

gmsh.initialize()
gmsh.model.add("t11")
mesh_size = 30

xpoints = np.array([gmsh.model.occ.addPoint(*X_a, mesh_size) for X_a in X_Ia])

for I_i in I_Fi:
    xpoints1 = xpoints[I_i]
    curves = [gmsh.model.occ.addLine(xpoints1[k], xpoints1[k + 1]) for k in range(len(xpoints1) - 1)] + [gmsh.model.occ.addLine(xpoints1[-1], xpoints1[0])]
    cl = gmsh.model.occ.addCurveLoop(curves)
    pl = gmsh.model.occ.addPlaneSurface([cl])

# Meshing ---------------------------------------------------- ------------------------------------------------

gmsh.model.occ.synchronize()

gmsh.model.mesh.generate(2)
# gmsh.model.mesh.recombine()
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
# gmsh.model.mesh.refine()

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()
elem_types, elem_tags, node_tags = gmsh.model.mesh.getElements()
print(gmsh.model.mesh.getElements())

gmsh.finalize()

(array([ 1,  3, 15]), [array([ 26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201,

### Flipping negative normals to unite all normals point (upwards)

In [5]:
X_Fia = X_Ia[I_Fi]
To_flip_F = np.cross(X_Fia[:, 1, :] - X_Fia[:, 0, :], X_Fia[:, 2, :] - X_Fia[:, 0, :])[:, 2] < 0 # all items where z of normal is negative
To_flip_F

array([False,  True,  True,  True, False, False, False,  True,  True,
        True, False, False, False,  True,  True,  True, False, False,
       False,  True,  True,  True, False, False, False,  True,  True,
        True, False, False])

In [6]:
I_Fi[To_flip_F] = np.flip(I_Fi[To_flip_F], axis=1)
I_Fi

array([[ 0,  1,  2],
       [ 4,  3,  0],
       [ 5,  1,  0],
       [ 3,  5,  0],
       [ 0,  2,  6],
       [ 0,  6,  4],
       [ 7,  8,  9],
       [10,  6,  7],
       [ 2,  8,  7],
       [ 6,  2,  7],
       [ 7,  9, 11],
       [ 7, 11, 10],
       [12,  6, 10],
       [14, 13, 12],
       [ 4,  6, 12],
       [13,  4, 12],
       [12, 10, 15],
       [12, 15, 14],
       [16, 17, 18],
       [ 5, 19, 16],
       [20, 17, 16],
       [19, 20, 16],
       [16, 18,  1],
       [16,  1,  5],
       [21, 19,  5],
       [23, 22, 21],
       [24, 19, 21],
       [22, 24, 21],
       [21,  5,  3],
       [21,  3, 23]])

In [7]:
I_Li = np.vstack(((I_Fi[:, (0, 1)]), (I_Fi[:, (0, 2)]), (I_Fi[:, (1, 2)])))
I_Li = np.sort(I_Li, axis=1)
I_Li = np.unique(I_Li, axis=0)
I_Li

array([[ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 0,  5],
       [ 0,  6],
       [ 1,  2],
       [ 1,  5],
       [ 1, 16],
       [ 1, 18],
       [ 2,  6],
       [ 2,  7],
       [ 2,  8],
       [ 3,  4],
       [ 3,  5],
       [ 3, 21],
       [ 3, 23],
       [ 4,  6],
       [ 4, 12],
       [ 4, 13],
       [ 5, 16],
       [ 5, 19],
       [ 5, 21],
       [ 6,  7],
       [ 6, 10],
       [ 6, 12],
       [ 7,  8],
       [ 7,  9],
       [ 7, 10],
       [ 7, 11],
       [ 8,  9],
       [ 9, 11],
       [10, 11],
       [10, 12],
       [10, 15],
       [12, 13],
       [12, 14],
       [12, 15],
       [13, 14],
       [14, 15],
       [16, 17],
       [16, 18],
       [16, 19],
       [16, 20],
       [17, 18],
       [17, 20],
       [19, 20],
       [19, 21],
       [19, 24],
       [21, 22],
       [21, 23],
       [21, 24],
       [22, 23],
       [22, 24]])

### Verify section

In [8]:
import bmcs_utils.api as bu
import k3d

X_Lia = X_Ia[I_Li]
path_2a = X_Lia[0, ...]

path = path_2a
points = np.array([[2, 2, 0], [-2, 2, 0], [-2, -2, 0], [2, -2, 0]])

points = bu.Extruder.transform_first_contour(path, points, adapt_dimensions=True)

print('path=', path)
print('points=', points)

plot = k3d.plot()

mesh = k3d.lines(points.astype(np.float32), [[0, 1], [1, 2], [2, 3], [3, 0]], color=0xc73737, indices_type='segment', width=0.5)
plot += mesh

mesh = k3d.lines(path.astype(np.float32), [[0, 1]], color=0, width=0.5)
plot += mesh
plot

path= [[ 0.  0.  0.]
 [50. 50.  0.]]
points= [[-1.4142135  1.4142135 -2.       ]
 [ 1.4142135 -1.4142135 -2.       ]
 [ 1.4142135 -1.4142135  2.       ]
 [-1.4142135  1.4142135  2.       ]]


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [9]:
def get_rot_matrix_around_vector(v, deg_angle):
    deg_angle = np.deg2rad(deg_angle)
    
    c = np.cos(deg_angle)
    s = np.sin(deg_angle)
    v_norm = v / np.sqrt(sum(v * v))

    # See: Rotation matrix from axis and angle (https://en.wikipedia.org/wiki/Rotation_matrix)
    cross_product_matrix = np.cross(v_norm, np.identity(v_norm.shape[0]) * -1)
    return c * np.identity(3) + s * cross_product_matrix + (1 - c) * np.outer(v_norm, v_norm)

In [10]:
from scipy.spatial.transform import Rotation
def rotate_points_around_vector(cs_points, vector, deg_angle, center_of_rotation=None):
    if center_of_rotation is None:
        cs_points_center = np.sum(cs_points, axis=0)/cs_points.shape[0]
    else:
        cs_points_center = center_of_rotation
    cs_points_at_origin = cs_points - cs_points_center

    rot_around_path = get_rot_matrix_around_vector(vector, deg_angle)
    r = Rotation.from_matrix(rot_around_path)
    cs_points_at_origin = r.apply(cs_points_at_origin)

    cs_points = cs_points_at_origin + cs_points_center
    return cs_points

In [11]:
# Test rotation
cs_points = np.array([[1, 0, 0], [0, 2, 0], [-1, 0, 0]])
vec = np.array([[1, 1, 0], [2, 2, 0]])
cs_points = bu.Extruder.transform_first_contour(vec, cs_points, adapt_dimensions=True)
cs_points_rotated = rotate_points_around_vector(cs_points, vec[1, :] - vec[0, :], 90, center_of_rotation=(cs_points[0, :] + cs_points[2, :])/2)
cs_points_rotated

import k3d
plot = k3d.plot()

mesh = k3d.lines(cs_points.astype(np.float32), [[0, 1], [1, 2], [2, 0]], color=0x072aca, indices_type='segment', width=0.07)
plot += mesh

mesh = k3d.lines(cs_points_rotated.astype(np.float32), [[0, 1], [1, 2], [2, 0]], color=0xc73737, indices_type='segment', width=0.03)
plot += mesh

mesh = k3d.lines(vec, [[0, 1]], color=0, width=0.1)
plot += mesh
plot

C:\Users\hspartali\Anaconda3\envs\bmcs_env\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [12]:
diff_vec = path_2a[1, :] - path_2a[0, :]
path_2a[0, :] = path_2a[0, :] - 0.001 * diff_vec
path_2a[1, :] = path_2a[1, :] + 0.001 * diff_vec

In [19]:
# See https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorials/python/t11.py
# dimTag means tuple of (dimention, tag). Where tag is like an ID

import gmsh
import sys
import numpy as np
import bmcs_utils.api as bu
from scipy.spatial.transform import Rotation

thickness = 2
fold_thickness = 0.6

# Each dimension Dim has Tags refering to objects starting from 1, 
#  (a point is dim=1, a line, surface or more is dim=2 and a volume is dim=3)

gmsh.initialize()
gmsh.model.add("t11")
mesh_size = 0

# Adding tunnels to subtract later: ---------------------------------------------------------------
creases = []

X_Lia = X_Ia[I_Li]
lines_num = X_Lia.shape[0]

# lines_num = 30
for l in range(lines_num):
    path_2a = X_Lia[l, ...]
    
    # Pipe cross-section
    cs_points = np.array([[1.4, 0, 0], [0, thickness - fold_thickness, 0], [-1.4, 0, 0]])
    cs_points = bu.Extruder.transform_first_contour(path_2a, cs_points, adapt_dimensions=True)

    # Rotate the triangle around its base if it points down 90, 180 (90+90), 270 (180+90)
    # (for a square or circle, this is not needed)
    center_of_rotation = (cs_points[0, :] + cs_points[2, :])/2
    for angle in range(3):
        if np.any(cs_points[:, 2] < -0.001):
            cs_points = rotate_points_around_vector(cs_points, path_2a[1, :] - path_2a[0, :], 90, center_of_rotation)
        else:
            break

    points = []
    for point in cs_points:
        points.append(gmsh.model.occ.addPoint(*point, mesh_size))
    lines = [gmsh.model.occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.occ.addLine(points[-1], points[0])]
    cl = gmsh.model.occ.addCurveLoop(lines)
    pl = gmsh.model.occ.addPlaneSurface([cl])

    # Pipe path (wire)
    points = []
    points.append(gmsh.model.occ.addPoint(*path_2a[0, :], mesh_size))
    points.append(gmsh.model.occ.addPoint(*path_2a[1, :], mesh_size))
    line = gmsh.model.occ.addLine(points[0], points[1])
    wire = gmsh.model.occ.addWire(curveTags = [line])
    creases.append(gmsh.model.occ.addPipe(dimTags = [(2, pl)], wireTag=wire)[0])

vols = gmsh.model.occ.getEntities(dim=3)
gmsh.model.occ.fuse(vols, vols)

# Adding outer area of the pattern with extrusion: ------------------------------------------------

xpoints = np.array([gmsh.model.occ.addPoint(*X_a, mesh_size) for X_a in X_Ia])

wb_facets = []

for I_i in I_Fi:
    xpoints1 = xpoints[I_i]
    curves = [gmsh.model.occ.addLine(xpoints1[k], xpoints1[k + 1]) for k in range(len(xpoints1) - 1)] + [gmsh.model.occ.addLine(xpoints1[-1], xpoints1[0])]

    cl = gmsh.model.occ.addCurveLoop(curves)
    pl = gmsh.model.occ.addPlaneSurface([cl])
    wb_facets.append(pl)
    
# #     To generate quadrangles instead of triangles, we can simply add
#     gmsh.model.mesh.setRecombine(1, pl)

# pg = gmsh.model.addPhysicalGroup(dim = 3, tags=wb_facets, name='pg')
# print(pg)

# Extrude (extrude is already a volume or CAD object)
for wb_facet in wb_facets:
    ext = gmsh.model.occ.extrude(dimTags=[(2, wb_facet)], dx=0, dy=0, dz=2, numElements=[], heights=[], recombine=True)

vols = gmsh.model.occ.getEntities(dim=3)
# tess_block = gmsh.model.occ.fuse(vols[len(creases):], vols[len(creases):])
tess_block = gmsh.model.occ.fuse(vols[1:], vols[1:])

# gmsh.model.occ.getEntities(dim=3)
print('before cut')
# gmsh.model.occ.cut(tess_block[0], [creases[0], creases[1], creases[2]])

print('creases=', creases)
print('creases.size=', len(creases))
print('vols=', gmsh.model.occ.getEntities(dim=3))
print('vols.size=', len(gmsh.model.occ.getEntities(dim=3)))
print(tess_block[0])

# gmsh.model.occ.cut(tess_block[0], [creases[0]])

print('vols.size=', len(gmsh.model.occ.getEntities(dim=3)))

vols = gmsh.model.occ.getEntities(dim=3)

gmsh.model.occ.cut([vols[1]], [vols[0]])

# gmsh.model.occ.cut(tess_block[0], creases)
gmsh.model.occ.remove(dimTags=gmsh.model.occ.getEntities(dim=2), recursive=True)

print('cutted')
# max3DTag = gmsh.model.occ.getMaxTag(3)

# Meshing ---------------------------------------------------- ------------------------------------------------

gmsh.model.occ.synchronize()

# field = gmsh.model.mesh.field
# field.add("MathEval", 1)
# field.setString(1, "F", "1")
# field.setAsBackgroundMesh(1)

# # To generate quadrangles instead of triangles, we can simply add
# gmsh.model.mesh.setRecombine(2, pl)

# If we'd had several surfaces, we could have used the global option
# "Mesh.RecombineAll":
#
# gmsh.option.setNumber("Mesh.RecombineAll", 1)

# You can also set the subdivision step alone, with
#
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

# gmsh.model.mesh.generate(2)

# Note that you could also apply the recombination algorithm and/or the
# subdivision step explicitly after meshing, as follows:
#
print('before generate')
# gmsh.model.mesh.generate(2)
print('mesh generated')
# gmsh.model.mesh.recombine()
print('mesh recombine')
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
# gmsh.model.mesh.refine()

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()
    
# gmsh.write('test_wb.stl')
# gmsh.write('test_wb.step')

gmsh.finalize()

before cut
creases= [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (3, 21), (3, 22), (3, 23), (3, 24), (3, 25), (3, 26), (3, 27), (3, 28), (3, 29), (3, 30), (3, 31), (3, 32), (3, 33), (3, 34), (3, 35), (3, 36), (3, 37), (3, 38), (3, 39), (3, 40), (3, 41), (3, 42), (3, 43), (3, 44), (3, 45), (3, 46), (3, 47), (3, 48), (3, 49), (3, 50), (3, 51), (3, 52), (3, 53), (3, 54)]
creases.size= 54
vols= [(3, 1), (3, 2)]
vols.size= 2
[(3, 2)]
vols.size= 2
cutted
before generate
mesh generated
mesh recombine


In [20]:
# Volumes created from faces doesn't perform well for CAD boolean operations,
#  bettwer create CAD object, such as extrude or ready to use objects like box, sphere...
import gmsh
import math
import sys

gmsh.initialize()

gmsh.model.add("t16")

# Let's build the same model as in `t5.py', but using constructive solid
# geometry.

# We can log all messages for further processing with:
gmsh.logger.start()

# We first create two cubes:
# gmsh.model.occ.addBox(0, 0, 0, 1, 1, 1, 1)
# tunnels_vol = gmsh.model.occ.addBox(0.3, 0.3, 0.3, 0.5, 0.5, 0.5, 2)
# tunnels_vol = gmsh.model.occ.addSphere(1.3, 0, 0, 0.5)


points = []
points.append(gmsh.model.occ.addPoint(0, 0, 0))
points.append(gmsh.model.occ.addPoint(1, 0, 0))
points.append(gmsh.model.occ.addPoint(1, 1, 0))
points.append(gmsh.model.occ.addPoint(0, 1, 0))
lines = [gmsh.model.occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.occ.addLine(points[-1], points[0])]
cl = gmsh.model.occ.addCurveLoop(lines)
pl = gmsh.model.occ.addPlaneSurface([cl])
gmsh.model.occ.extrude(dimTags=[(2, pl)], dx=0, dy=0, dz=2, numElements=[], heights=[], recombine=True)


points = []
points.append(gmsh.model.occ.addPoint(1, 0, 0))
points.append(gmsh.model.occ.addPoint(2, 0, 0))
points.append(gmsh.model.occ.addPoint(2, 1, 0))
points.append(gmsh.model.occ.addPoint(1, 0.5, 0))
lines = [gmsh.model.occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.occ.addLine(points[-1], points[0])]
cl = gmsh.model.occ.addCurveLoop(lines)
pl = gmsh.model.occ.addPlaneSurface([cl])
gmsh.model.occ.extrude(dimTags=[(2, pl)], dx=0, dy=0, dz=2, numElements=[], heights=[], recombine=True)

points = []
points.append(gmsh.model.occ.addPoint(-0.2, 0.2, 0.2))
points.append(gmsh.model.occ.addPoint(0.5, 0.2, 0.2))
points.append(gmsh.model.occ.addPoint(0.5, 0.5, 0.2))
points.append(gmsh.model.occ.addPoint(-0.2, 0.5, 0.2))
lines = [gmsh.model.occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.occ.addLine(points[-1], points[0])]
cl = gmsh.model.occ.addCurveLoop(lines)
pl = gmsh.model.occ.addPlaneSurface([cl])
gmsh.model.occ.extrude(dimTags=[(2, pl)], dx=0, dy=0, dz=0.5, numElements=[], heights=[], recombine=True)

# Pipe:
#-------
# Pipe cross-section
points = []
points.append(gmsh.model.occ.addPoint(0, 0, 0))
points.append(gmsh.model.occ.addPoint(0.1, 0, 0))
points.append(gmsh.model.occ.addPoint(0.1, 0.3, 0))
points.append(gmsh.model.occ.addPoint(0, 0.3, 0))
lines = [gmsh.model.occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.occ.addLine(points[-1], points[0])]
cl = gmsh.model.occ.addCurveLoop(lines)
pl = gmsh.model.occ.addPlaneSurface([cl])

# Pipe path (wire)
points = []
points.append(gmsh.model.occ.addPoint(0, 0, 0))
points.append(gmsh.model.occ.addPoint(1, 1, 1))
line = gmsh.model.occ.addLine(points[0], points[1])
wire = gmsh.model.occ.addWire(curveTags = [line])
pipe = gmsh.model.occ.addPipe(dimTags = [(2, cl)], wireTag=wire)

# We apply a boolean difference to create the "cube minus one eigth" shape:
gmsh.model.occ.cut([(3, 1)], [(3, 3)])

# There's still following boolean:
# intersect (cut opposite) and fragment (union with keeping intersecting parts) and fuse (union)

gmsh.model.occ.synchronize()

gmsh.model.mesh.generate(3)

gmsh.write("t16.msh")

# Additional examples created with the OpenCASCADE geometry kernel are available
# in `t18.py', `t19.py' and `t20.py', as well as in the `examples/api'
# directory.

# Inspect the log:
log = gmsh.logger.get()
print("Logger has recorded " + str(len(log)) + " lines")
gmsh.logger.stop()

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()


Logger has recorded 225 lines


In [15]:
wbt = WBTessellation5PV2(wb_cell = WBCell5ParamV2(a = 50, eta=1, zeta=1, gamma = np.pi/2-1e-3), n_x=2, n_y=2, wireframe_width=1)
wbt.interact()

W: 24.999972454328425
T_rho: 0.2499998495367709
T_sigma: 0.2499998495850007
sol_P2_t_2: 2179.5192269945655
sol_Q2_u_2: 648.9385818409175
Ana. solution: [-3.1385107  -3.14067502]


In [2]:
wb = WBCell4Param()
wb.interact()

C:\Users\hspartali\Anaconda3\envs\bmcs_env\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


In [3]:
wb.X_Ia

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+03,  9.99999875e+02,  5.00000042e-01],
       [-1.00000000e+03,  9.99999875e+02,  5.00000042e-01],
       [ 1.00000000e+03, -9.99999875e+02,  5.00000042e-01],
       [-1.00000000e+03, -9.99999875e+02,  5.00000042e-01],
       [ 9.99999500e+02,  0.00000000e+00,  9.99999833e-01],
       [-9.99999500e+02,  0.00000000e+00,  9.99999833e-01]])

In [4]:
wb.I_Fi

array([[0, 1, 2],
       [0, 3, 4],
       [0, 1, 5],
       [0, 5, 3],
       [0, 2, 6],
       [0, 6, 4]])

In [30]:
X_Ia[:, -1]+100

array([100., 100., 100., 100., 100., 100., 100.])

In [58]:
vertices

array([[ 0.00000000e+00,  2.50000000e+01,  0.00000000e+00],
       [ 0.00000000e+00,  2.49506682e+01, -1.56976299e+00],
       [ 0.00000000e+00,  2.48028675e+01, -3.13333084e+00],
       ...,
       [ 2.18602151e+02,  1.86021506e+01,  3.13333084e+00],
       [ 2.18713001e+02,  1.87130012e+01,  1.56976299e+00],
       [ 2.18750000e+02,  1.87500000e+01, -1.60812265e-14]])

In [5]:
import k3d

thickness = 100
X_Ia = wb.X_Ia
I_Fi = wb.I_Fi

plot = k3d.plot()

mesh1 = k3d.mesh(X_Ia, I_Fi, side='double', color=0x999999)
plot+= mesh1
mesh1 = k3d.mesh(X_Ia, I_Fi, side='double', wireframe=True, color=0)
plot+= mesh1

# z_X_Ia = np.copy(X_Ia)
# z_X_Ia[:, -1] += thickness
# mesh2 = k3d.mesh(z_X_Ia, I_Fi, side='double', color=0x999999)
# plot+= mesh2

plot

C:\Users\hspartali\Anaconda3\envs\bmcs_env\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
C:\Users\hspartali\Anaconda3\envs\bmcs_env\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [66]:
# Write to obj file
f = open('wb_3d_print.obj', 'w')
f.write('# Vertices: (' + str(len(vertices)) + ')\n')
for v in X_Ia:
    f.write('v ' + str(v)[1:-1] + '\n')
f.write('\n# Tri Facets: (' + str(len(indices)) + ')\n')
for ind in I_Fi + 1:
    f.write('f ' + str(ind)[1:-1] + '\n')
f.close()